In [3]:
%run startup.py

In [4]:
%%javascript
$.getScript('./assets/js/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# A Decision Tree of Observable Operators

## Part 1: NEW Observables.

> source: http://reactivex.io/documentation/operators.html#tree.  
> (transcribed to RxPY 1.5.7, Py2.7 / 2016-12, Gunther Klessinger, [axiros](http://www.axiros.com))  

**This tree can help you find the ReactiveX Observable operator you’re looking for.**  

<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

## Usage

There are no configured behind the scenes imports or code except [`startup.py`](./edit/startup.py), which defines output helper functions, mainly:

- `rst, reset_start_time`: resets a global timer, in order to have use cases starting from 0.
- `subs(observable)`: subscribes to an observable, printing notifications with time, thread, value


All other code is explicitly given in the notebook.  
Since all initialisiation of tools is in the first cell, you always have to run the first cell after ipython kernel restarts.  
**All other cells are autonmous.**

In the use case functions, in contrast to the official examples we simply use **`rand`** quite often (mapped to `randint(0, 100)`), to demonstrate when/how often observable sequences are generated and when their result is buffered for various subscribers.  
*When in doubt then run the cell again, you might have been "lucky" and got the same random.*

### RxJS
The (bold printed) operator functions are linked to the [official documentation](http://reactivex.io/documentation/operators.html#tree) and created roughly analogous to the **RxJS** examples. The rest of the TOC lines links to anchors within the notebooks. 

### Output
When the output is not in marble format we display it like so:

```
new subscription on stream 276507289 

   3.4  M [next]    1.4: {'answer': 42}
   3.5 T1 [cmpl]    1.6: fin
   
```
where the lines are syncronously `print`ed as they happen.  "M" and "T1" would be thread names ("M" is main thread).  
For each use case in `reset_start_time()` (alias `rst`), a global timer is set to 0 and we show the offset to it, in *milliseconds* & with one decimal value and also the offset to the start of stream subscription. In the example 3.4, 3.5 are millis since global counter reset, while 1.4, 1.6 are offsets to start of subscription.


# I want to create a **NEW** Observable...

## ... that emits a particular item: **[just](http://reactivex.io/documentation/operators/just.html) **

In [9]:
reset_start_time(reactivex.just)
stream = reactivex.just({'answer': rand()})
disposable = subs(stream)
sleep(0.5)
disposable = subs(stream) # same answer
# all stream ops work, its a real stream:
disposable = subs(stream.pipe(ops.map(lambda x: x.get('answer', 0) * 2)))




========== return_value ==========

module reactivex
def return_value(
    value: _T, scheduler: Optional[abc.SchedulerBase] = None
):
    Alias for :func:`reactivex.return_value`.
--------------------------------------------------------------------------------

   1.8     M New subscription (29074) on stream 280528954
   2.1     M [next]    0.3: {'answer': 104} -> 29074
   2.1     M [cmpl]    0.3: fin -> 29074

 506.1     M New subscription (21323) on stream 280528954
 506.8     M [next]    0.6: {'answer': 104} -> 21323
 506.8     M [cmpl]    0.6: fin -> 21323

 507.4     M New subscription (47661) on stream 279325172
 507.7     M [next]    0.2: 208 -> 47661
 507.7     M [cmpl]    0.2: fin -> 47661


## ..that was returned from a function *called at subscribe-time*: **[start](http://reactivex.io/documentation/operators/start.html)**

In [11]:
print('There is a little API difference to RxJS, see Remarks:\n')
rst(reactivex.start)

def f():
    log('function called')
    return rand()

stream = reactivex.start(func=f)
d = subs(stream)
d = subs(stream)

header("Exceptions are handled correctly (an observable should never except):")

def breaking_f():    
    return 1 / 0

stream = reactivex.start(func=breaking_f)
d = subs(stream)
d = subs(stream)



# startasync: only in python3 and possibly here(?) http://www.tornadoweb.org/en/stable/concurrent.html#tornado.concurrent.Future
#stream = O.start_async(f)
#d = subs(stream)


There is a little API difference to RxJS, see Remarks:



========== start ==========

module reactivex
def start(
    func: Callable[[], _T], scheduler: Optional[abc.SchedulerBase] = None
):
    Invokes the specified function asynchronously on the specified
    scheduler, surfacing the result through an observable sequence.

    .. marble::
        :alt: start

        [ start(lambda i: return 4) ]
        -4-|
          -4-|

    Note:
        The function is called immediately, not during the subscription
        of the resulting sequence. Multiple subscriptions to the
        resulting sequence can observe the function's result.

    Example:
        >>> res = reactivex.start(lambda: pprint('hello'))
        >>> res = reactivex.start(lambda: pprint('hello'), rx.Scheduler.timeout)

    Args:
        func: Function to run asynchronously.
        scheduler: [Optional] Scheduler to run the function on. If
            not specified, defaults to an instance of
            :class:`Timeout

## ..that was returned from an Action, Callable, Runnable, or something of that sort, called at subscribe-time: **[from](http://reactivex.io/documentation/operators/from.html)**

In [13]:
rst(reactivex.from_iterable)
def f():
    log('function called')
    return rand()
# aliases: reactivex.from_, reactivex.from_list
# 1.: From a tuple:
stream = reactivex.from_iterable((1,2,rand()))
d = subs(stream)
# d = subs(stream) # same result

# 2. from a generator
gen = (rand() for j in range(3))
stream = reactivex.from_iterable(gen)
d = subs(stream)





========== from_iterable ==========

module reactivex
def from_iterable(
    iterable: Iterable[_T], scheduler: Optional[abc.SchedulerBase] = None
):
    Converts an iterable to an observable sequence.

    .. marble::
        :alt: from_iterable

        [   from_iterable(1,2,3)    ]
        ---1--2--3--|


    Example:
        >>> reactivex.from_iterable([1,2,3])

    Args:
        iterable: An Iterable to change into an observable sequence.
        scheduler: [Optional] Scheduler instance to schedule the values on.
            If not specified, the default is to use an instance of
            :class:`CurrentThreadScheduler
            <reactivex.scheduler.CurrentThreadScheduler>`.

    Returns:
        The observable sequence whose elements are pulled from the
        given iterable sequence.
--------------------------------------------------------------------------------

   3.6     M New subscription (18926) on stream 279066055
   3.8     M [next]    0.2: 1 -> 18926
   3.8     M

In [19]:
rst(reactivex.from_callback)
# in my words: In the on_next of the subscriber you'll have the original arguments,
# potentially objects, e.g. user original http requests.
# i.e. you could merge those with the result stream of a backend call to
# a webservice or db and send the request.response back to the user then.

def g(f, a, b):
    f(a, b)
    log('called f')
stream = reactivex.from_callback(lambda a, b, f: g(f, a, b))('fu', 'bar')
d = subs(stream.pipe(ops.delay(200)))
# d = subs(stream.delay(200)) # does NOT work




========== from_callback ==========

module reactivex
def from_callback(
    func: Callable[..., Callable[..., None]],
    mapper: Optional[typing.Mapper[Any, Any]] = None,
):
    Converts a callback function to an observable sequence.

    Args:
        func: Function with a callback as the last argument to
            convert to an Observable sequence.
        mapper: [Optional] A mapper which takes the arguments
            from the callback to produce a single item to yield on
            next.

    Returns:
        A function, when executed with the required arguments minus
        the callback, produces an Observable sequence with a single
        value of the arguments to the callback as a list.
--------------------------------------------------------------------------------

   5.9     M New subscription (55311) on stream 279325332
   6.5     M called f


## ...after a specified delay: **[timer](http://reactivex.io/documentation/operators/timer.html)**

In [21]:
rst()
# start a stream of 0, 1, 2, .. after 200 ms, with a delay of 100 ms:
stream = reactivex.timer(200, 100).time_interval()\
    .map(lambda x: 'val:%s dt:%s' % (x.value, x.interval))\
    .take(3)
d = subs(stream, name='observer1')
# intermix directly with another one
d = subs(stream, name='observer2')



========== True ==========



AttributeError: 'Observable' object has no attribute 'time_interval'

## ...that emits a sequence of items repeatedly: **[repeat](http://reactivex.io/documentation/operators/repeat.html) **

In [26]:
rst(ops.repeat)
# repeat is over *values*, not function calls. Use generate or create for function calls!
subs(ops.repeat(20))

header('do while:')
l = []
def condition(x):
    l.append(1)
    return True if len(l) < 2 else False
stream = reactivex.just(42).do_while(condition)
d = subs(stream)





========== repeat ==========

module reactivex.operators
def repeat(
    repeat_count: Optional[int] = None,
):
    Repeats the observable sequence a specified number of times.
    If the repeat count is not specified, the sequence repeats
    indefinitely.

    .. marble::
        :alt: repeat

        -1--2-|
        [    repeat(3)     ]
        -1--2--1--2--1--2-|


    Examples:
        >>> repeated = repeat()
        >>> repeated = repeat(42)
    Args:
        repeat_count: Number of times to repeat the sequence. If not
        provided, repeats the sequence indefinitely.

    Returns:
        An operator function that takes an observable sources and
        returns an observable sequence producing the elements of the
        given sequence repeatedly.
--------------------------------------------------------------------------------

   2.1     M New subscription (40310) on stream 281144037


AttributeError: 'function' object has no attribute 'subscribe'

45537.3    T9 [next] 300013.7: ['fu', 'bar'] -> 23120
45539.6    T9 [cmpl] 300015.8: fin -> 23120


## ...from scratch, with custom logic and cleanup (calling a function again and again): **[create](http://reactivex.io/documentation/operators/create.html) **

In [28]:
rx = reactivex.create
rst(rx)

def f(obs):
    # this function is called for every observer
    obs.on_next(rand())
    obs.on_next(rand())
    obs.on_completed()
    def cleanup():
        log('cleaning up...')
    return cleanup
stream = reactivex.create(f).pipe(ops.delay(200)) # the delay causes the cleanup called before the subs gets the vals
d = subs(stream)
d = subs(stream)




sleep(0.5)
rst(title='Exceptions are handled nicely')
l = []
def excepting_f(obs):
    for i in range(3):
        l.append(1)
        obs.on_next('%s %s (observer hash: %s)' % (i, 1. / (3 - len(l)), hash(obs) ))
    obs.on_completed()

stream = reactivex.create(excepting_f)
d = subs(stream)
d = subs(stream)




rst(title='Feature or Bug?')
print('(where are the first two values?)')
l = []
def excepting_f(obs):
    for i in range(3):
        l.append(1)
        obs.on_next('%s %s (observer hash: %s)' % (i, 1. / (3 - len(l)), hash(obs) ))
    obs.on_completed()

stream = reactivex.create(excepting_f).pipe(ops.delay(100))
d = subs(stream)
d = subs(stream)
# I think its an (amazing) feature, preventing to process functions results of later(!) failing functions




========== create ==========

module reactivex
def create(subscribe: typing.Subscription[_T]):
    Creates an observable sequence object from the specified
        subscription function.

    .. marble::
        :alt: create

        [     create(a)    ]
        ---1---2---3---4---|

    Args:
        subscribe: Subscription function.

    Returns:
        An observable sequence that can be subscribed to via the given
        subscription function.
--------------------------------------------------------------------------------

   1.5     M New subscription (22997) on stream 281172788
   2.0     M [err ]    0.6: f() takes 1 positional argument but 2 were given -> 22997

   2.2     M New subscription (55140) on stream 281172788
   2.5     M [err ]    0.3: f() takes 1 positional argument but 2 were given -> 55140


========== Exceptions are handled nicely ==========


   1.4     M New subscription (73023) on stream 279372192
   1.9     M [err ]    0.4: excepting_f() takes 1 positional

In [10]:
rx = O.generate
rst(rx)
"""The basic form of generate takes four parameters:

the first item to emit
a function to test an item to determine whether to emit it (true) or terminate the Observable (false)
a function to generate the next item to test and emit based on the value of the previous item
a function to transform items before emitting them
"""
def generator_based_on_previous(x): return x + 1.1
def doubler(x): return 2 * x
d = subs(rx(0, lambda x: x < 4, generator_based_on_previous, doubler))



========== generate ==========

module rx.linq.observable.generate
@extensionclassmethod(Observable)
def generate(cls, initial_state, condition, iterate, result_mapper, scheduler=None):
    Generates an observable sequence by running a state-driven loop
    producing the sequence's elements, using the specified scheduler to
    send out observer messages.

    1 - res = reactivex.Observable.generate(0,
        lambda x: x < 10,
        lambda x: x + 1,
        lambda x: x)
    2 - res = reactivex.Observable.generate(0,
        lambda x: x < 10,
        lambda x: x + 1,
        lambda x: x,
        Rx.Scheduler.timeout)

    Keyword arguments:
    initial_state -- Initial state.
    condition -- Condition to terminate generation (upon returning False).
    iterate -- Iteration step function.
    result_mapper -- Selector function for results produced in the
        sequence.
    scheduler -- [Optional] Scheduler on which to run the generator loop.
        If not provided, defaults to 

In [29]:
rx = reactivex.generate_with_relative_time
rst(rx)
stream = rx(1, lambda x: x < 4, lambda x: x + 1, lambda x: x, lambda t: 100)
d = subs(stream)




========== generate_with_relative_time ==========

module reactivex
def generate_with_relative_time(
    initial_state: _TState,
    condition: typing.Predicate[_TState],
    iterate: typing.Mapper[_TState, _TState],
    time_mapper: Callable[[_TState], typing.RelativeTime],
):
    Generates an observable sequence by iterating a state from an
    initial state until the condition fails.

    .. marble::
        :alt: generate_with_relative_time

        [generate_with_relative_time()]
        -1-2-3-4-|

    Example:
        >>> res = reactivex.generate_with_relative_time(
            0, lambda x: True, lambda x: x + 1, lambda x: 0.5
        )

    Args:
        initial_state: Initial state.
        condition: Condition to terminate generation (upon returning
            :code:`False`).
        iterate: Iteration step function.
        time_mapper: Time mapper function to control the speed of
            values being produced each iteration, returning relative times, i.e.
           

TypeError: generate_with_relative_time() takes 4 positional arguments but 5 were given

## ...for each observer that subscribes OR according to a condition at subscription time: **[defer / if_then](http://reactivex.io/documentation/operators/defer.html) **

In [12]:
rst(O.defer)
# plural! (unique per subscription)
streams = O.defer(lambda: O.just(rand()))
d = subs(streams)
d = subs(streams) # gets other values - created by subscription!



========== defer ==========

module rx.linq.observable.defer
@extensionclassmethod(Observable)
def defer(cls, observable_factory):
    Returns an observable sequence that invokes the specified factory
    function whenever a new observer subscribes.

    Example:
    1 - res = reactivex.Observable.defer(lambda: reactivex.Observable.from_([1,2,3]))

    Keyword arguments:
    :param types.FunctionType observable_factory: Observable factory function
        to invoke for each observer that subscribes to the resulting sequence.

    :returns: An observable sequence whose observers trigger an invocation
    of the given observable factory function.
    :rtype: Observable
--------------------------------------------------------------------------------

   2.7     M New subscription on stream 274475969
   3.4     M [next]    0.6: 38
   3.5     M [cmpl]    0.7: fin

   4.4     M New subscription on stream 274475969
   4.9     M [next]    0.4: 77
   5.2     M [cmpl]    0.7: fin


In [13]:
# evaluating a condition at subscription time in order to decide which of two streams to take.
rst(O.if_then)
cond = True
def should_run():
    return cond
streams = O.if_then(should_run, O.return_value(43), O.return_value(56))
d = subs(streams)

log('condition will now evaluate falsy:')
cond = False
streams = O.if_then(should_run, O.return_value(43), O.return_value(rand()))
d = subs(streams)
d = subs(streams)



========== if_then ==========

module rx.linq.observable.ifthen
@extensionclassmethod(Observable)
def if_then(cls, condition, then_source, else_source=None, scheduler=None):
    Determines whether an observable collection contains values.

    Example:
    1 - res = reactivex.Observable.if(condition, obs1)
    2 - res = reactivex.Observable.if(condition, obs1, obs2)
    3 - res = reactivex.Observable.if(condition, obs1, scheduler=scheduler)

    Keyword parameters:
    condition -- {Function} The condition which determines if the
        then_source or else_source will be run.
    then_source -- {Observable} The observable sequence or Promise that
        will be run if the condition function returns true.
    else_source -- {Observable} [Optional] The observable sequence or
        Promise that will be run if the condition function returns False.
        If this is not provided, it defaults to reactivex.empty
    scheduler -- [Optional] Scheduler to use.

    Returns an observable {

## ...that emits a sequence of integers: **[range](http://reactivex.io/documentation/operators/range.html) **

In [14]:
rst(O.range)
d = subs(O.range(0, 3))



========== range ==========

module rx.linq.observable.range
@extensionclassmethod(Observable)
def range(cls, start, count, scheduler=None):
    Generates an observable sequence of integral numbers within a
    specified range, using the specified scheduler to send out observer
    messages.

    1 - res = reactivex.Observable.range(0, 10)
    2 - res = reactivex.Observable.range(0, 10, rx.Scheduler.timeout)

    Keyword arguments:
    start -- The value of the first integer in the sequence.
    count -- The number of sequential integers to generate.
    scheduler -- [Optional] Scheduler to run the generator loop on. If not
        specified, defaults to Scheduler.current_thread.

    Returns an observable sequence that contains a range of sequential
    integral numbers.
--------------------------------------------------------------------------------

   2.9     M New subscription on stream 274475905
   3.7     M [next]    0.4: 0
   4.3     M [next]    1.0: 1
   4.6     M [next]    

### ...at particular intervals of time: **[interval](http://reactivex.io/documentation/operators/interval.html) **

(you can `.publish()` it to get an easy "hot" observable)

In [47]:
rst(reactivex.interval)
d = subs(reactivex.interval(period=100).pipe(ops.time_interval(),ops.map(lambda x, v: '%(interval)s %(value)s',ItemGetter(x)).take(3)))



========== interval ==========

module reactivex
def interval(
    period: typing.RelativeTime, scheduler: Optional[abc.SchedulerBase] = None
):
    Returns an observable sequence that produces a value after each period.

    .. marble::
        :alt: interval

        [  interval()   ]
        ---1---2---3---4--->

    Example:
        >>> res = reactivex.interval(1.0)

    Args:
        period: Period for producing the values in the resulting sequence
            (specified as a :class:`float` denoting seconds or an instance of
            :class:`timedelta`).
        scheduler:  Scheduler to run the interval on. If not specified, an
            instance of :class:`TimeoutScheduler <reactivex.scheduler.TimeoutScheduler>`
            is used.

    Returns:
        An observable sequence that produces a value after each period.
--------------------------------------------------------------------------------


NameError: name 'x' is not defined

### ...after a specified delay (see timer)

## ...that completes without emitting items: **[empty](http://reactivex.io/documentation/operators/empty-never-throw.html) **

In [48]:
rst(reactivex.empty)
d = subs(reactivex.empty())



========== empty ==========

module reactivex
def empty(scheduler: Optional[abc.SchedulerBase] = None):
    Returns an empty observable sequence.

    .. marble::
        :alt: empty

        [     empty()     ]
        --|

    Example:
        >>> obs = reactivex.empty()

    Args:
        scheduler: [Optional] Scheduler instance to send the termination call
            on. By default, this will use an instance of
            :class:`ImmediateScheduler <reactivex.scheduler.ImmediateScheduler>`.

    Returns:
        An observable sequence with no elements.
--------------------------------------------------------------------------------

   5.9     M New subscription (71158) on stream 281172869
   6.7     M [cmpl]    0.7: fin -> 71158


## ...that does nothing at all: **[never](http://reactivex.io/documentation/operators/empty-never-throw.html) **

In [49]:
rst(reactivex.never)
d = subs(reactivex.never())



========== never ==========

module reactivex
def never():
    Returns a non-terminating observable sequence, which can be used
    to denote an infinite duration (e.g. when using reactive joins).

    .. marble::
        :alt: never

        [     never()     ]
        -->

    Returns:
        An observable sequence whose observers will never get called.
--------------------------------------------------------------------------------

   4.1     M New subscription (27752) on stream 281229809


## ...that excepts: **[throw](http://reactivex.io/documentation/operators/empty-never-throw.html) **

In [54]:
rst(reactivex.Observer.on_error)
d = subs(reactivex.on_error(error=ZeroDivisionError))



========== on_error ==========

module reactivex.observer.observer
def on_error(self, error: Exception):
    Notify the observer that an exception has occurred.

        Args:
            error: The error that occurred.
--------------------------------------------------------------------------------


AttributeError: module 'reactivex' has no attribute 'on_error'